In [25]:
import csv
import glob
import os
import re

In [26]:
from ete3 import Tree
from tqdm import tqdm

In [27]:
from main.tree import fitch, get_node_age

In [28]:
OUT_DIR = "data/homology/ss/out/"

In [29]:
tree_no_dist = list()

In [30]:
ss_gene = {5: dict(), 3: dict()}
ss_exon = {5: dict(), 3: dict()}

In [31]:
with open("data/ref/homo_sapiens/annot/out/meta_exon.csv", 'r') as f:
    reader = csv.reader(f)
    next(reader)
    
    for row in reader:
        gene, gene_func, chrom, meta_beg, meta_end, strand, meta_kind, make_up = row
        
        if gene_func != "protein_coding":
            continue
        
        strand = int(strand == "+")
        meta_exon = f"{meta_kind}={chrom}:{meta_beg}-{meta_end}:{strand}"
        
        for exon in make_up.split(","):
            kind, *pos = re.split('[=-]', exon)
            
            if kind == "SE":
                continue
            
            beg, end = map(int, pos)
            beg -= 1
            
            ss5 = None
            ss3 = None
            
            if kind == "FE":
                ss5 = end if strand else beg
                
            if kind == "IE":
                ss5 = end if strand else beg
                ss3 = beg if strand else end
            
            if kind == "LE":
                ss3 = beg if strand else end
            
            if ss5:
                ss5 = f"{chrom}:{ss5}:{strand}"
                
                ss_gene[5][ss5] = gene
                ss_exon[5][ss5] = meta_exon
            
            if ss3:
                ss3 = f"{chrom}:{ss3}:{strand}"
                
                ss_gene[3][ss3] = gene
                ss_exon[3][ss3] = meta_exon

In [32]:
TREE = Tree("data/tree/time_tree.nw", format=1)

In [33]:
for i, node in enumerate(TREE.traverse('postorder')):
    if not node.is_leaf():
        node.name = f"inner_node_{i}"

In [34]:
NODE_AGE = get_node_age(TREE)

In [35]:
tree_no_dist = list()

In [36]:
with open(f"{OUT_DIR}/origin_new.csv", 'w') as f_0:
    writer = csv.writer(f_0)
    writer.writerow([
        "gene",
        "meta_exon",
        "p",
        "ss",
        "hsap_score",
        "est_gain_event/s",
        "est_loss_event/s",
        "pos_group",
        "neg_group",
        "tree_size",
        "min_changes",
    ])
    
    for i, fp in enumerate(glob.iglob("data/homology/ss/scores/*.csv")):
        chrom, p = os.path.basename(fp).removesuffix(".csv").split("_")

        print(chrom)
        p = int(p)
        
        with open(fp, 'r') as f_1:
            tot = sum(1 for _ in f_1)
        
        with open(fp, 'r') as f_2:
            reader = csv.reader(f_2)
            _, *names = next(reader)
            
            
            for row in tqdm(reader, total=tot):
                pos, *scores = row
                
                name_state = dict()
                hsap_score = None
                
                for name, score in zip(names, scores):
                    if score != "-":
                        state = int(float(score) > 0)
                        name_state[name] = state
                        
                        if name == "homo_sapiens":
                            hsap_score = score
                
                # Ignore cases where the H. sap. SS is non-positive.
                if name_state['homo_sapiens'] == 0:
                    continue
                
                # We have nothing to estimate when the leaf states are homogenous.
                states = set(name_state.values())
                if states == {1}:
                    continue
                
                tree = TREE.copy(method='deepcopy')
                tree.prune(name_state.keys())
                
                for leaf in tree:
                    leaf.state = {name_state[leaf.name]}  # `fitch` operates on `set` states.
            
                min_changes, annot_trees = fitch(tree)
                
                no_trees = len(annot_trees)
                tree_no_dist.append(no_trees)
                
                if no_trees != 1:
                    continue
                else:
                    annot_tree, = annot_trees
                
                gain_events = set()
                loss_events = set()
                
                for node in annot_tree.traverse('postorder'):
                    if node.gain:
                        age = str(NODE_AGE[node.name])
                        gain_events.add(age)
                    
                    if node.loss:
                        age = str(NODE_AGE[node.name])
                        loss_events.add(age)
                
                # `pos` is the 100-bp range that we search, so I update it to
                # be the position of the (annotated) exon-intron junciton.
                chrom, *pos = re.split('[:-]', pos)
                beg, end, strand = map(int, pos)
                
                ss = f"{chrom}:{beg + 50}:{strand}"
                
                gene = ss_gene[p][ss]
                meta_exon = ss_exon[p][ss]
                    
                writer.writerow([
                    gene,
                    meta_exon,
                    p,
                    ss,
                    hsap_score,
                    ";".join(sorted(gain_events)) if gain_events else "-",
                    ";".join(sorted(loss_events)) if loss_events else "-",
                    ";".join([name for name, state in name_state.items() if state == 1]),
                    ";".join([name for name, state in name_state.items() if state == 0]),
                    len(tree),
                    min_changes
                ])

18


100%|█████████▉| 3399/3400 [00:06<00:00, 531.10it/s]


6


100%|█████████▉| 10068/10069 [00:26<00:00, 376.39it/s]


21


...

100%|█████████▉| 7042/7043 [00:08<00:00, 827.51it/s] 


14


100%|█████████▉| 6188/6189 [00:12<00:00, 506.49it/s]


12


100%|█████████▉| 12440/12441 [00:18<00:00, 658.25it/s]


In [37]:
with open(f"{OUT_DIR}/tree_no_dist.txt", 'w') as f:
    f.write("\n".join([str(x) for x in tree_no_dist]))